In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import grid2op
from grid2op.PlotGrid import PlotMatplot
from lightsim2grid import LightSimBackend

c:\Users\Ernest\.conda\envs\l2rpn-test\lib\site-packages\grid2op\Backend\pandaPowerBackend.py:32: UserWarning: Numba cannot be loaded. You will gain possibly massive speed if installing it by 
	c:\Users\Ernest\.conda\envs\l2rpn-test\python.exe -m pip install numba

  warnings.warn(


In [17]:
import os
os.chdir("../")
os.getcwd()

'e:\\github_clone\\L2RPN-rte-5'

In [2]:
env_name = "rte_case5_example"  # or any other name.
env = grid2op.make(env_name, test=True)

c:\Users\Ernest\.conda\envs\l2rpn-test\lib\site-packages\grid2op\MakeEnv\Make.py:438: UserWarning: You are using a development environment. This environment is not intended for training agents. It might not be up to date and its primary use if for tests (hence the "test=True" you passed as argument). Use at your own risk.
  warnings.warn(_MAKE_DEV_ENV_WARN)


In [9]:
LEARNING_ITERATION = 10
EVAL_EPISODE = 1
MAX_EVAL_STEPS = 10

In [11]:
from grid2op.gym_compat import GymEnv
import grid2op
from gymnasium import Env
from gymnasium.utils.env_checker import check_env
try:
    from lightsim2grid import LightSimBackend
    bk_cls = LightSimBackend
except ImportError as exc:
    print(f"Error: {exc} when importing faster LightSimBackend")
    from grid2op.Backend import PandaPowerBackend
    bk_cls = PandaPowerBackend
    
env_name = "rte_case5_example"
training_env = grid2op.make(env_name, test=True)  # we put "test=True" in this notebook because...
# it's a notebook to explain things. Of course, do not put "test=True" if you really want
# to train an agent...
gym_env = GymEnv(training_env)

c:\Users\Ernest\.conda\envs\l2rpn-test\lib\site-packages\grid2op\MakeEnv\Make.py:438: UserWarning: You are using a development environment. This environment is not intended for training agents. It might not be up to date and its primary use if for tests (hence the "test=True" you passed as argument). Use at your own risk.
  warnings.warn(_MAKE_DEV_ENV_WARN)


In [12]:
gym_env.action_space

Dict('change_bus': MultiBinary(21), 'change_line_status': MultiBinary(8), 'set_bus': Box(-1, 2, (21,), int32), 'set_line_status': Box(-1, 1, (8,), int32))

In [13]:
from grid2op.gym_compat import DiscreteActSpace
gym_env.action_space = DiscreteActSpace(training_env.action_space)
gym_env.action_space

Discrete(166)

In [14]:
obs, reward, terminated, truncated, info = gym_env.step(53)  # perform action labeled 0
obs

OrderedDict([('a_ex',
              array([7.1869745e+02, 4.3956577e+02, 4.5736687e+01, 5.6552578e+01,
                     4.5497098e+02, 5.4125762e-12, 5.4125762e-12, 8.9389944e+00],
                    dtype=float32)),
             ('a_or',
              array([703.2866   , 422.33826  ,  39.433754 ,  51.658306 , 462.6719   ,
                       7.7699723,   7.7699723,   9.35489  ], dtype=float32)),
             ('actual_dispatch', array([0., 0.], dtype=float32)),
             ('attention_budget', array([0.], dtype=float32)),
             ('current_step', array([1])),
             ('curtailment', array([0., 0.], dtype=float32)),
             ('curtailment_limit', array([1., 1.], dtype=float32)),
             ('curtailment_limit_effective', array([1., 1.], dtype=float32)),
             ('day', 6),
             ('day_of_week', 6),
             ('delta_time', array([5.], dtype=float32)),
             ('duration_next_maintenance', array([0, 0, 0, 0, 0, 0, 0, 0])),
             ('gen_m

In [15]:
from stable_baselines3 import PPO
nn_model = PPO(env=gym_env,
               learning_rate=1e-3,
               policy="MultiInputPolicy",
               policy_kwargs={"net_arch": [100, 100, 100]},
               n_steps=2,
               batch_size=8,
               verbose=True,
               )

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\Ernest\.conda\envs\l2rpn-test\lib\site-packages\stable_baselines3\ppo\ppo.py:155: UserWarning: You have specified a mini-batch size of 8, but because the `RolloutBuffer` is of size `n_steps * n_envs = 2`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 2
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2 and n_envs=1)
  warnings.warn(


In [16]:
nn_model.learn(total_timesteps=LEARNING_ITERATION)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 2        |
|    ep_rew_mean     | 4.5      |
| time/              |          |
|    fps             | 0        |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2        |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1.5        |
|    ep_rew_mean          | 2.25       |
| time/                   |            |
|    fps                  | 0          |
|    iterations           | 2          |
|    time_elapsed         | 7          |
|    total_timesteps      | 4          |
| train/                  |            |
|    approx_kl            | 0.09049064 |
|    clip_fraction        | 0.5        |
|    clip_range           | 0.2        |
|    entropy_loss         | -5.11      |
|    explained_variance   | -0.01      |
|    learning_rate        | 0.001      |
|   

In [19]:
import grid2op
from grid2op.Runner import Runner
from grid2op.Agent import RandomAgent
env = grid2op.make(env_name, test=True)
NB_EPISODE = 10  # assess the performance for 10 episodes, for example
NB_CORE = 2  # do it on 2 cores, for example
PATH_SAVE = "agents_log"  # and store the results in the "agents_log" folder
runner = Runner(**env.get_params_for_runner(), agentClass=RandomAgent)
runner.run(nb_episode=NB_EPISODE, nb_process=NB_CORE, path_save=PATH_SAVE)

c:\Users\Ernest\.conda\envs\l2rpn-test\lib\site-packages\grid2op\MakeEnv\Make.py:438: UserWarning: You are using a development environment. This environment is not intended for training agents. It might not be up to date and its primary use if for tests (hence the "test=True" you passed as argument). Use at your own risk.
  warnings.warn(_MAKE_DEV_ENV_WARN)


[('c:\\Users\\Ernest\\.conda\\envs\\l2rpn-test\\lib\\site-packages\\grid2op\\data\\rte_case5_example\\chronics\\00',
  '00',
  8.319282531738281,
  3,
  2016),
 ('c:\\Users\\Ernest\\.conda\\envs\\l2rpn-test\\lib\\site-packages\\grid2op\\data\\rte_case5_example\\chronics\\02',
  '02',
  0.0,
  1,
  2016),
 ('c:\\Users\\Ernest\\.conda\\envs\\l2rpn-test\\lib\\site-packages\\grid2op\\data\\rte_case5_example\\chronics\\04',
  '04',
  19.28192138671875,
  6,
  2016),
 ('c:\\Users\\Ernest\\.conda\\envs\\l2rpn-test\\lib\\site-packages\\grid2op\\data\\rte_case5_example\\chronics\\06',
  '06',
  20.38006591796875,
  6,
  2016),
 ('c:\\Users\\Ernest\\.conda\\envs\\l2rpn-test\\lib\\site-packages\\grid2op\\data\\rte_case5_example\\chronics\\08',
  '08',
  44.322593688964844,
  7,
  2016),
 ('c:\\Users\\Ernest\\.conda\\envs\\l2rpn-test\\lib\\site-packages\\grid2op\\data\\rte_case5_example\\chronics\\01',
  '01',
  39.95808792114258,
  7,
  2016),
 ('c:\\Users\\Ernest\\.conda\\envs\\l2rpn-test\\lib\\